In [1]:
import pandas as pd
from datetime import date, timedelta


In [12]:
flow_df_banff = pd.read_csv('bow-river-level-at-banff.csv',parse_dates=[['Date', 'Time']])
flow_df_calgary = pd.read_csv('bow-river-flow-target-feature.csv', parse_dates=[['Date', 'Time']])

In [13]:
flow_df_calgary = flow_df_calgary.resample('60min', on='Date_Time').mean()

In [38]:
#flow_df_calgary = flow_df_calgary.set_index('Date_Time')

In [36]:
flow_df_calgary = flow_df_calgary[flow_df_calgary.index.month > 3]
flow_df_calgary = flow_df_calgary[flow_df_calgary.index.month < 10]
flow_df_calgary = flow_df_calgary[flow_df_calgary.index.year >1999]
flow_df_calgary = flow_df_calgary[flow_df_calgary.index.year <2020]
flow_df_calgary_parsed =flow_df_calgary.fillna(method='backfill')

In [41]:
flow_df_calgary_parsed.to_pickle('../parsed/dataframe1.pkl')